In [41]:
import pandas_gbq
from google.cloud import bigquery

client = bigquery.Client()
query = """
    SELECT shipping_latitude, shipping_longitude from `no-maintenance`.shopify.orders
    WHERE shipping_latitude IS NOT NULL and shipping_longitude IS NOT NULL and json_extract_scalar(shipping_address, '$.country_code') = 'US'
     and shipping_address IS NOT NULL
    """
df = pandas_gbq.read_gbq(query, project_id=client.project)


Downloading: 100%|██████████|


In [46]:
df

,shipping_latitude,shipping_longitude,cluster
0,40.718914,-74.002525,5
1,34.125803,-118.259983,10
2,39.006625,-76.899261,11
3,34.059252,-118.280178,10
4,34.125803,-118.259983,10
...,...,...,...
4514,39.322237,-74.605478,5
4515,34.198869,-118.474672,10
4516,42.636768,-71.656698,13
4517,41.230553,-111.975305,14


In [47]:
from sklearn.cluster import KMeans

# create kmeans model/object
kmeans = KMeans(
    init="random",
    n_clusters=16,
    n_init=10,
    max_iter=300,
    random_state=42
)

In [48]:
# do clustering
kmeans.fit(df)
# save results
labels = kmeans.labels_

In [49]:
df['cluster'] = labels
# display the number of mamber each clustering
_clusters = df.groupby('cluster').size()
print(_clusters)

cluster
0       63
1      840
2      251
3      402
4      116
5       27
6       25
7      162
8      277
9      365
10     182
11     192
12    1142
13     334
14     136
15       5
dtype: int64


In [50]:
import folium
colors = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
    "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
    "#bcbd22", "#17becf", "#aec7e8", "#ffbb78",
    "#98df8a", "#ff9896", "#c5b0d5", "#c49c94"
]
lat = df.iloc[0]['shipping_latitude']
lng = df.iloc[0]['shipping_longitude']

In [51]:
map = folium.Map(location=[lat, lng], zoom_start=12)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["shipping_latitude"], row["shipping_longitude"]],
        radius=12,
        weight=2,
        fill=True,
        fill_color=colors[int(row["cluster"])],
        color=colors[int(row["cluster"])]
    ).add_to(map)
map